# Imports

In [ ]:
import numpy as np
import pandas as pd
import zarr
import holoviews as hv
import datashader as ds
from holoviews.operation.datashader import (
    aggregate,
    datashade,
    dynspread,
    shade,
    regrid,
)
from holoviews.operation import decimate
from holoviews.streams import Stream, param
import matplotlib.pyplot as plt
from matplotlib.colors import hex2color
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, clear_output
import skimage
import skimage.morphology
import peakutils
import scipy.stats
import scipy.interpolate
import functools
import operator
from functools import partial, reduce
import itertools
from itertools import zip_longest
from more_itertools import rstrip
from collections import Counter, defaultdict
from collections.abc import Mapping, Sequence
from bokeh.models import WheelZoomTool
from bokeh.io import push_notebook, show, output_notebook

# from bokeh.layouts import row
# from bokeh.plotting import figure
import tqdm
from tqdm import tnrange, tqdm, tqdm_notebook
from copy import copy, deepcopy
import warnings

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from processing import *
from trench_detection import *
from trench_detection import _label_for_trenches
from trench_segmentation import *
from util import *
from ui import *

In [ ]:
%load_ext line_profiler
hv.notebook_extension("bokeh")
# renderer = hv.renderer('bokeh')
%matplotlib inline
tqdm.monitor_interval = 0

# Config

In [ ]:
# warnings.filterwarnings('ignore', 'Conversion of the second argument of issubdtype')
warnings.filterwarnings("ignore", category=FutureWarning)
# warnings.filterwarnings('default', category=FutureWarning)

In [ ]:
# store = zarr.DirectoryStore('/home/jqs1/scratch/fidelity/171018/171018.zarr')
store = zarr.DirectoryStore(
    "/home/jqs1/scratch/fidelity/171214/transcriptionerror_timelapse.zarr"
)
root_group = zarr.open_group(store=store)

# Compute

In [ ]:
diag_pos = tree()
trench_points_pos = {
    pos: fail_silently(
        lambda: get_trenches(
            root_group["raw"][str(pos)][0, 30], diagnostics=diag_pos[pos]
        )
    )
    for pos in tnrange(350)
}

In [ ]:
none_pos = [k for k, v in trench_points_pos.items() if v is None]

In [ ]:
{k: v[""] for k, v in diag_pos.items() if k not in none_pos}

In [ ]:
none_pos

In [ ]:
diag = tree()
get_trenches(root_group["raw"][str(3)][0, 30], diagnostics=diag)

In [ ]:
diag["trench_rotation"]

In [ ]:
display_plot_browser(diag)

In [ ]:
frame_stream = FrameStream()
display(frame_browser(root_group["quantized"], frame_stream))
display_plot_browser(lambda t, v: diag_pos[v] if v in diag_pos else None, frame_stream)

In [ ]:
big_image_viewer(root_group["quantized"])

In [ ]:
%%time
sharpness = np.array(
    [image_sharpness(frame_series[i]) for i in range(frame_series.shape[0])]
)

# Memory debugging

In [ ]:
import cloudpickle

In [ ]:
import resource

resource.getrusage(resource.RUSAGE_SELF).ru_maxrss / 1024

In [ ]:
resource?

In [ ]:
resource.getrusage(resource.RUSAGE_SELF)

In [ ]:
import objgraph

In [ ]:
objgraph.most_common_types()

In [ ]:
len(b)

In [ ]:
import gc

gc.collect()

In [ ]:
from pympler import asizeof

In [ ]:
diag_pos[0]["label_1"]["trench_ends"].keys()

In [ ]:
c = cloudpickle.dumps(diag_pos[0]["label_1"]["trench_ends"])

In [ ]:
with open("foo", "wb") as f:
    cloudpickle.dump(diag_pos, f)

In [ ]:
len(c)

In [ ]:
print(asizeof.asized(diag_pos[0], detail=5).format())

In [ ]:
%store -r

In [ ]:
%store trench_points_pos
%store diag_pos

# Image viewers

In [ ]:
# big_image_viewer(root_group['quantized'])

In [ ]:
def overlay_image_viewer(frames):
    frame_stream = FrameStream()
    slider_box = frame_browser(frames, frame_stream)
    channels_box, display_settings_stream = multichannel_selector(frames)

    def image_callback(t, v, channel_enabled, channel_colors):
        channel_imgs = [
            frames[v, c, t, :, :] for c in range(frames.shape[1]) if channel_enabled[c]
        ]
        img = composite_channels(channel_imgs, select(channel_colors, channel_enabled))
        viewer = hv.RGB(img[::-1], bounds=(0, 0, img.shape[1], img.shape[0]))
        return viewer

    image = hv.DynamicMap(
        image_callback, streams=[frame_stream, display_settings_stream]
    )
    image = regrid(image)
    image = image.opts(plot={"width": 500, "height": 500})
    output = widgets.Output()
    box = widgets.VBox([widgets.HBox([channels_box, slider_box]), output])
    display(box)
    with output:
        display(image)
    return None


overlay_image_viewer(frames_z)

In [ ]:
%%opts Image (cmap='viridis')
def scrubber_callback(t, y):
    t = int(t)
    v_line = hv.VLine(t + 0.5)(style={"color": "white", "alpha": 0.3, "line_width": 2})
    t_label = hv.Text(t + kymo.shape[1] / 20, kymo.shape[0] / 20, "t={:}".format(t))(
        style={"color": "white", "text_alpha": 0.8}
    )
    h_line = hv.HLine(y)(style={"color": "white", "alpha": 0.3, "line_width": 2})
    return v_line * t_label * h_line


def cross_section_callback(t, y):
    y = int(y)
    h_line = hv.HLine(y)(style={"color": "white", "alpha": 0.3, "line_width": 2})
    return h_line


def sharpness_scrubber_callback(t):
    t = int(t)
    v_line = hv.VLine(t + 0.5)(style={"color": "black", "alpha": 0.3, "line_width": 2})
    return v_line


def trench_thumbnail_callback(img_series, t):
    t = int(t)
    img = get_trench_thumbnail(img_series[t], trench_points, trench_idx)
    # TODO: don't know why calling hv.Image.opts(plot={'invert_axes': True}) doesn't work
    # also switched HLine to VLine in cross_section, above
    thumb = hv.Image(
        img[::-1], bounds=(0, 0, img.shape[1], img.shape[0]), kdims=["x2", "y"]
    )
    # return thumb
    # return thumb.opts(plot={'invert_axes': False, 'width': max_thumbnail_width})#, 'xaxis': None, 'yaxis': None})
    return thumb.opts(
        plot={"invert_axes": False, "invert_yaxis": True, "width": 200}
    )  # , 'xaxis': None, 'yaxis': None})
    # return thumb.opts(plot={'invert_axes': False, 'invert_yaxis': True})


pointerx = hv.streams.PointerX(x=0).rename(x="t")
pointery = hv.streams.PointerY(y=0)

# BOUNDS: (left, bottom, top, right)
trench_idx = 15
img_series = frame_series_k1
bbox_ul, bbox_lr = get_trench_bbox(
    trench_points, trench_idx, *get_img_limits(img_series[0])
)
kymo = extract_kymograph(
    img_series, trench_points[0][trench_idx], trench_points[1][trench_idx]
)
kymo_img = hv.Image(kymo[::-1], bounds=(0, 0, kymo.shape[1], kymo.shape[0])).opts(
    plot={"width": 700, "height": 300, "invert_yaxis": True}
)
# kymo_img = hv.Raster(kymo).opts(plot={'width': 700, 'height': 300, 'yaxis': 'left'})
scrubber_line = hv.DynamicMap(scrubber_callback, streams=[pointerx, pointery])
cross_section_line = hv.DynamicMap(cross_section_callback, streams=[pointerx, pointery])
trench_thumbnail_img = hv.DynamicMap(
    partial(trench_thumbnail_callback, img_series), streams=[pointerx]
)  # .opts(plot={'invert_axes': True})
sharpness_plot = hv.Curve(sharpness, kdims=["x"], vdims=["s"]).opts(
    plot={"width": 700, "height": 100}
)
sharpness_scrubber = hv.DynamicMap(sharpness_scrubber_callback, streams=[pointerx])
# pointery.source = kymo_img
# SEE: http://holoviews.org/reference/elements/bokeh/Distribution.html
# kymo_img * scrubber_line << (trench_thumbnail_img * cross_section_line) << (sharpness_plot * sharpness_scrubber)

KymoOverlayStream = Stream.define("KymoOverlayStream", overlay_enabled=True)
kymo_overlay_stream = KymoOverlayStream()
kymo_overlay_button = widgets.ToggleButton(description="Overlay", value=True)


def update_kymo_overlay(change):
    kymo_overlay_stream.event(overlay_enabled=change["new"])


kymo_overlay_button.observe(update_kymo_overlay, names="value")


def get_thumbnail_overlay(t, overlay_enabled):
    # because of https://github.com/ioam/holoviews/issues/1388, overlay_enabled must be True initially
    x0 = trench_points[0][trench_idx]
    x1 = trench_points[1][trench_idx]
    trench_line = hv.Curve([x0 - bbox_ul, x1 - bbox_ul]).opts(
        style={"color": "white", "alpha": 0.5, "line_width": 1.5},
        plot={"yaxis": None, "shared_axes": True},
    )
    overlays = []
    if overlay_enabled:
        overlays.append(trench_line)
    else:
        pass  # overlays = []#line.opts(style={'alpha': 0.0, 'color': 'red'})
    return hv.Overlay(overlays)


thumbnail_overlay = hv.DynamicMap(
    get_thumbnail_overlay, streams=[pointerx, kymo_overlay_stream]
)  # .opts(plot={'invert_yaxis': True})

display(kymo_overlay_button)
(
    kymo_img * scrubber_line
    << (trench_thumbnail_img * thumbnail_overlay * cross_section_line)
    << (sharpness_plot * sharpness_scrubber)
)
# kymo_img * scrubber_line << (trench_thumbnail_img * cross_section_line) << (sharpness_plot * sharpness_scrubber)
# trench_thumbnail_img * thumbnail_overlay
# trench_thumbnail_img * thumbnail_overlay * cross_section_line

In [ ]:
def trench_peaks_callback(t, y):
    t = int(t)
    overlays = [hv.Curve(kymo[:, t]).opts(plot={"width": 500})]
    overlays.extend(
        [
            hv.Points([(x, kymo[int(x), t])]).opts(style={"size": 6})
            for x in kymo_endpoints[t]
        ]
    )
    return hv.Overlay(overlays)


hv.DynamicMap(trench_peaks_callback, streams=[pointerx, pointery])

In [ ]:
%%time
def find_kymograph_cell_endpoints(kymograph, thresh=0.2, min_dist=3):
    endpoints = []
    for t in range(kymograph.shape[1]):
        idxs = peakutils.indexes(kymograph[:, t], thres=thresh, min_dist=min_dist)
        xs = idxs
        # xs = peakutils.interpolate(np.arange(kymograph.shape[0]), kymograph[:,t], ind=idxs)
        endpoints.append(xs)
    return endpoints


kymo_endpoints = find_kymograph_cell_endpoints(kymo, thresh=0.2, min_dist=5)

In [ ]:
for t in range(kymo.shape[1] // 5):
    plt.plot(kymo[220:, t])

# Old

In [ ]:
# %%opts Image (cmap='viridis')
img_series = frame_series
kymo = extract_kymograph(
    img_series, trench_points[0][trench_idx], trench_points[1][trench_idx]
)
kymo_img = hv.Image(kymo, bounds=(0, 0, kymo.shape[1], kymo.shape[0])).opts(
    plot={"width": 700, "height": 300}
)
trench_thumbnail_img = hv.DynamicMap(
    partial(trench_thumbnail, img_series), streams=[pointerx]
)
(
    kymo_img.opts(plot={"invert_axes": True}) * scrubber_line
    + trench_thumbnail_img * cross_section_line
)

In [ ]:
# handle out-of-bounds/negative t/x values
# focus quality score
# show focus quality as a bar above kymograph
# accurate horizontal line position on thumbnail
# draw cross-section line through thumbnail
# synchronize zoom/pan of multiple kymograph viewers
# 3-up thumbnail viewer with crosshairs/endpoint correspondences (links) synchronized with kymograph viewer
# clicking through a track advances 3-up by one frame
# if you press END it finishes track and rewinds to the first unfinished track
# automatic tracking: always identify bottom-most endpoint?
# can we work out all other correspondences from this? (look at t=14)